In [63]:
#importing reqired libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, confusion_matrix
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

data=pd.read_csv('train.csv')

#removing ID column
data.drop(columns=['ID'],axis=1,inplace=True)
data.head()

#checking for class imbalance 
def class_imbalance(target):
    class_values = (target.value_counts()/target.value_counts().sum())*100
    return class_values
class_imbalance(data['Claim'])

# def square(data):
#     data=data**2
#     return data


#label encoding the data
le = LabelEncoder()
# Function that auto encodes any dataframe column of type category or object.
def dummyEncode(dataset):
        
        columnsToEncode = list(dataset.select_dtypes(include=['category','object']))
        le = LabelEncoder()
        for feature in columnsToEncode:
            try:
                dataset[feature] = le.fit_transform(dataset[feature])
            except:
                print('Error encoding '+feature)
        return dataset
data = dummyEncode(data)


#predictors for training
X = data.iloc[:,:-1]

# Target
y = data.iloc[:,-1]


sc = StandardScaler()
X = sc.fit_transform(X)


def random_forrest(dataframe,target):
    
    x_train,x_val,y_train,y_val = train_test_split(dataframe,target, test_size=0.3, random_state=42)
    
    # Applying Smote on train data for dealing with class imbalance
    smote = SMOTE()
    X_sm, y_sm =  smote.fit_sample(x_train, y_train)
    
    global rfc
    rfc = RandomForestClassifier(n_estimators=200, criterion='gini', max_depth=50, min_samples_split=4, max_features='auto',random_state=0)
    rfc.fit(x_train, y_train)
    y_pred=rfc.predict(x_val)
    precision=precision_score(y_val,y_pred)
#     print(confusion_matrix(y_val, y_pred))
    return precision

#trainning
precision = random_forrest(X,y)    
print('score is:',precision)


#testing function
def prediction(test):
    y_pred = rfc.predict(test)
    
    return y_pred

test=pd.read_csv('test.csv')

# Storing the Id column
Id = test[['ID']]

# Preprocessed Test File
test.drop('ID',1,inplace=True)
test.head()
#label encoder
test = dummyEncode(test)
test.head()

#predicting on test file
y_pred = pd.DataFrame(prediction(test),columns=['Claim']) 
# print(y_pred['Claim'].value_counts())
submission = pd.concat([Id,y_pred['Claim']],1)
submission.to_csv('submission.csv',index=False)

score is: 0.8536895674300254
